# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

I try to find out in 2006, which cities have TOP 10 population move in ?
So I need combine immigration date and iata_code which have location relative to iata code.
I choose immigration data from udacity data set sample
and extrat IATA code from website:https://www.nationsonline.org/oneworld/IATA_Codes/airport_code_list.htm
and get the city name from IATA code that in the immigration. 
But I found above "airport_code_csv" is not include metranpolitan code like "NYC". 
and i search iata_code in wiki website:"https://en.wikipedia.org/wiki/List_of_airports_by_IATA_code:_"
add get all the new iata code and combine all of them to one table:
this table have three columns: iata_id,airport,location
### 'iata_id' is 3-alpha letter , string type.

### 'airport' is airport name, string type.

### 'location' is country name or city name, string type.
using python script :"get_iata_code_from_wiki.py" that written by myself.

 ### I extracted i94yr: as year,i94mon: as month,i94cit: as from_city,i94port: as to_city, admnum: as total_immi_num.


 
 


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [1]:
import pandas as pd
fname_i = 'immigration_data_sample.csv'
df = pd.read_csv(fname_i)
df.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [2]:
# create the sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
#read and load golbal land temperature by city dataset in csv format
df_sp_immi  = spark.read.format("csv").option("header", "true").load("immigration_data_sample.csv")
df_sp_immi.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [3]:
### I extracted i94yr: as year,i94mon: as month,i94cit: as from_city,i94port: as to_city, admnum: as total_immi_num.

In [4]:
# drop duplicates row
df_immi_spark=df_sp_immi.dropDuplicates()


In [5]:
df_immi_spark.count()

1000

In [6]:
#read and load ""airport-codes_csv.csv""
df_iata_default  = spark.read.format("csv").option("header", "true").load("airport-codes_csv.csv")
df_iata_default.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [7]:
df_iata_default.show()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [8]:
df_iata_default.dropDuplicates().count()

55075

In [9]:
#read and load golbal land temperature by city dataset in csv format
df_iata_wiki  = spark.read.format("csv").option("header", "true").load("iata_code1104.csv")
df_iata_wiki.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- ICAO_CODE: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- Location: string (nullable = true)



In [10]:
df_iata_wiki.show()

+---------+---------+--------------------+--------------------+
|IATA_CODE|ICAO_CODE|             airport|            Location|
+---------+---------+--------------------+--------------------+
|      AAA|     NTGA|        Anaa Airport|Anaa, Tuamotus, F...|
|      AAB|     YARY|    Arrabury Airport|Arrabury, Queensl...|
|      AAC|     HEAR|El Arish Internat...|     El Arish, Egypt|
|      AAD|     null|       Adado Airport|Adado (Cadaado), ...|
|      AAE|     DABB|Rabah Bitat Airpo...|     Annaba, Algeria|
|      AAF|     KAAF|Apalachicola Regi...|Apalachicola, Flo...|
|      AAG|     SSYA|     Arapoti Airport|Arapoti, Paraná, ...|
|      AAH|     EDKA|   Merzbrück Airport|Aachen, North Rhi...|
|      AAI|     SWRA|     Arraias Airport|Arraias, Tocantin...|
|      AAJ|     SMCA|     Cayana Airstrip|  Awaradam, Suriname|
|      AAK|     NGUK|     Aranuka Airport|   Aranuka, Kiribati|
|      AAL|     EKYT|     Aalborg Airport|    Aalborg, Denmark|
|      AAM|     FAMD|   Mala Mala Airpor

In [11]:
df_iata_wiki.dropDuplicates()
df_iata_wiki.count()

12872

# At first,I use pandas to identify data quality issues,like missing values,duplicate data.


In [12]:
# transform spark dataframe to pandas dataframe
# because airport-codes_csv.csv is not good data , so i choose my own data iata_code1104.csv from wiki website, it is clearer and better than airport-codes_csv.csv
pd_iata_wiki = df_iata_wiki.toPandas()
# drop all the duplicate "iata-code"
drop_d=pd_iata_wiki.drop_duplicates('IATA_CODE')
# transform pandas dataframe back to spark dataframe
sp_df_wiki=spark.createDataFrame(drop_d)
# count the total row number
sp_df_wiki.count()

9010

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

#### Fact Table
1.immigration population in world.
              _c0 AS id,
              i94yr AS year, 
              i94mon AS month,
              i94cit AS from_city,
              i94port AS iata_id,
              CAST(admnum AS bigint) AS total_immi_num 
#### Dimension Tables
2.location
iata_id,airport,location




### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [26]:
# build the immigration data pipeline to create the data model.
df_immi_spark.createOrReplaceTempView("immi")
out=spark.sql('''SELECT DISTINCT
              _c0 AS id,
              i94yr AS year, 
              i94mon AS month,
              i94cit AS from_city,
              i94port AS to_city,
              CAST(admnum AS bigint) AS total_immi_num 
              FROM immi''')
out.write.parquet("./output/immi0.parquet")

In [13]:
sp_df_wiki.createOrReplaceTempView("location_view")

In [14]:

location= spark.sql('''select DISTINCT IATA_CODE AS iata_id, 
                    airport,
                    Location AS location
                    FROM location_view
                
                    ''')
location.show()

+-------+--------------------+--------------------+
|iata_id|             airport|            location|
+-------+--------------------+--------------------+
|    AGO|Magnolia Municipa...|Magnolia, Arkansa...|
|    BDT|   Gbadolite Airport|Gbadolite, Democr...|
|    BGA|Palonegro Interna...|Bucaramanga, Colo...|
|    BIO|      Bilbao Airport|Bilbao, Basque Co...|
|    BWN|Brunei Internatio...|Bandar Seri Begaw...|
|    CLJ|Cluj-Napoca Inter...|Cluj-Napoca, Romania|
|    CYL|     Coyoles Airport|   Coyoles, Honduras|
|    DMN|Deming Municipal ...|Deming, New Mexic...|
|    FUG|Fuyang Xiguan Air...|Fuyang, Anhui, China|
|    GHF| Giebelstadt Airport|Giebelstadt, Bava...|
|    IFJ|  Ísafjörður Airport| Ísafjörður, Iceland|
|    KMZ|       Kaoma Airport|       Kaoma, Zambia|
|    KNZ|     Kéniéba Airport|       Kéniéba, Mali|
|    MIZ|  Mainoru Airport[1]|Mainoru, Northern...|
|    MNN|Marion Municipal ...|Marion, Ohio, Uni...|
|    MQL|     Mildura Airport|Mildura, Victoria...|
|    MRF|Mar

In [15]:
location.printSchema()

root
 |-- iata_id: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- location: string (nullable = true)



In [24]:
#add iata code from wiki 12870 codes. to "./output/location.parquet" 
location.write.mode("Append").parquet("./output/location.parquet")

In [25]:
# load iata model date from parquet. and save to location_view.
location_view=spark.read.parquet("./output/location.parquet")
location_view.createOrReplaceTempView("location_view")

In [17]:
# base on the immigration model ,get top 10 immigration population cities
immi_num=spark.read.parquet("./output/immi2.parquet")
immi_num.createOrReplaceTempView("immi_table")
sorted_city= spark.sql('''

                    SELECT to_city,
                    SUM(total_immi_num) AS total
                    FROM immi_table
                    GROUP BY to_city
                    ORDER BY total DESC

                    ''')
sorted_city.show(10)

+-------+--------------+
|to_city|         total|
+-------+--------------+
|    NYC|10846125450235|
|    MIA| 8922547681245|
|    LOS| 6965071195933|
|    SFR| 3591412487120|
|    CHI| 3020403711605|
|    ORL| 2736974886745|
|    HOU| 2728330237345|
|    ATL| 2669580186622|
|    NEW| 2589234201821|
|    HHW| 2461775306030|
+-------+--------------+
only showing top 10 rows



In [18]:
# create tempview sorted city population
sorted_city.createOrReplaceTempView("sorted_view")

In [26]:
# top 10 immigration cities which immigration move in
answer=spark.sql('''SELECT DISTINCT
                    s.to_city,
                    l.location,
                    l.airport,
                    s.total
                    FROM sorted_view as s
                    JOIN location_view AS l
                    WHERE s.to_city=l.iata_id
                    ORDER BY s.total DESC
                    ''')     


In [27]:
# top 10 cities immigration moving into.
answer.show(10)

+-------+--------------------+--------------------+--------------+
|to_city|            location|             airport|         total|
+-------+--------------------+--------------------+--------------+
|    NYC|New York City, Ne...|  metropolitan area2|10846125450235|
|    MIA|Miami, Florida, U...|Miami Internation...| 8922547681245|
|    LOS|      Lagos, Nigeria|Murtala Muhammed ...| 6965071195933|
|    CHI|Chicago, Illinois...|  metropolitan area2| 3020403711605|
|    ORL|Orlando, Florida,...|Orlando Executive...| 2736974886745|
|    HOU|Houston, Texas, U...|William P. Hobby ...| 2728330237345|
|    ATL|Atlanta, Georgia,...|Hartsfield–Jackso...| 2669580186622|
|    NEW|New Orleans, Loui...|   Lakefront Airport| 2589234201821|
|    WAS|Washington, D.C.,...|  metropolitan area1| 1765844128444|
|    DAL|Dallas, Texas, Un...|   Dallas Love Field| 1317476983970|
+-------+--------------------+--------------------+--------------+
only showing top 10 rows



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

# Perform quality checks here
# As you can see above, sorted_city top 10 citits is different from the answer top 10 cities

### from immigration data : top 10 is 
+-------+--------------+
|to_city|         total|
+-------+--------------+
|    NYC|10846125450235|
|    MIA| 8922547681245|
|    LOS| 6965071195933|
|    SFR| 3591412487120|
|    CHI| 3020403711605|
|    ORL| 2736974886745|
|    HOU| 2728330237345|
|    ATL| 2669580186622|
|    NEW| 2589234201821|
|    HHW| 2461775306030|
+-------+--------------+

### but after combine the two tables: fact table and demension. the result is :
+-------+--------------------+--------------------+--------------+
|to_city|            location|             airport|         total|
+-------+--------------------+--------------------+--------------+
|    NYC|New York City, Ne...|  metropolitan area2|10846125450235|
|    MIA|Miami, Florida, U...|Miami Internation...| 8922547681245|
|    LOS|      Lagos, Nigeria|Murtala Muhammed ...| 6965071195933|
|    CHI|Chicago, Illinois...|  metropolitan area2| 3020403711605|
|    ORL|Orlando, Florida,...|Orlando Executive...| 2736974886745|
|    HOU|Houston, Texas, U...|William P. Hobby ...| 2728330237345|
|    ATL|Atlanta, Georgia,...|Hartsfield–Jackso...| 2669580186622|
|    NEW|New Orleans, Loui...|   Lakefront Airport| 2589234201821|
|    WAS|Washington, D.C.,...|  metropolitan area1| 1765844128444|
|    DAL|Dallas, Texas, Un...|   Dallas Love Field| 1317476983970|
+-------+--------------------+--------------------+--------------+

the result is wrong ,because there is miss city "SFR" and "HHW". So more iata code data need to be collected, and then Append to "./output/location.parquet".


In [32]:
# add more data into location.parquet , and then clean the duplicate row.

backup_iata=spark.read.parquet("./output/iata_table2.parquet")


In [44]:
backup_iata.select('iata_id','airport','location').where(backup_iata['iata_id']=='SFR').show()
# So i load the backup iata code date from iata_table2.parquet and append to "./output/location.parquet"
backup_iata.write.mode("Append").parquet("./output/location.parquet")

+-------+--------------------+--------+
|iata_id|             airport|location|
+-------+--------------------+--------+
|    SFR|San Fernando Airport|      US|
+-------+--------------------+--------+



In [49]:
# do the clean data process again , clean duplicated row in iata_id, because treat "iata code" as primary key
# read "./output/location.parquet" to spark data frame 
update_location= spark.read.parquet("./output/location.parquet")
dropd = update_location.dropDuplicates()
# transfer the spark dataframe to pandas dataframe 
df=dropd.toPandas()
# Using dataframe methods to clean duplicates iata code
iata_code=df.drop_duplicates('iata_id')



In [51]:
#transfer the Pandas data frame back to spark data frame
sp_df_location=spark.createDataFrame(iata_code)
sp_df_location.count()

9422

In [52]:
sp_df_location.printSchema()

root
 |-- iata_id: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- location: string (nullable = true)



In [53]:
sp_df_location.createOrReplaceTempView("updated_location_view")

In [54]:
# top 10 immigration cities which immigration move in
final_answer=spark.sql('''SELECT DISTINCT
                    s.to_city,
                    l.location,
                    l.airport,
                    s.total
                    FROM sorted_view as s
                    JOIN updated_location_view AS l
                    WHERE s.to_city=l.iata_id
                    ORDER BY s.total DESC
                    ''')     


In [55]:
final_answer.show(10)

+-------+--------------------+--------------------+--------------+
|to_city|            location|             airport|         total|
+-------+--------------------+--------------------+--------------+
|    NYC|New York City, Ne...|  metropolitan area2|10846125450235|
|    MIA|Miami, Florida, U...|Miami Internation...| 8922547681245|
|    LOS|      Lagos, Nigeria|Murtala Muhammed ...| 6965071195933|
|    SFR|                  US|San Fernando Airport| 3591412487120|
|    CHI|Chicago, Illinois...|  metropolitan area2| 3020403711605|
|    ORL|                  US|Orlando Executive...| 2736974886745|
|    HOU|Houston, Texas, U...|William P. Hobby ...| 2728330237345|
|    ATL|Atlanta, Georgia,...|Hartsfield–Jackso...| 2669580186622|
|    NEW|New Orleans, Loui...|   Lakefront Airport| 2589234201821|
|    WAS|Washington, D.C.,...|  metropolitan area1| 1765844128444|
+-------+--------------------+--------------------+--------------+
only showing top 10 rows



### As you can see above, the 10th city is wrong ,it must be "HHW", not "WAS" Washington, D.C.
But I haven't found the result when I searched "HHW" in iata code in wiki websit,
So I infer the "HHW" is wrong data ,  maybe it is "KHHW "in ICAO CODE. It represents "KHHW Stan Stamper Municipal Airport (FAA: HHW) Hugo, Oklahoma, United States"


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

## Fact table: immigration data set, from udacity data set sample.

### I extracted 
### i94yr: as year, can be cast as datetime type
### i94mon: as month, can be cast as datetime type
### i94cit: as from_city, "3 digit Numeric code ISO 3166 codes for each country." 
### i94port: as to_city, "alpha-3 code character,IATA CODE" string type
### admnum: as total_immi_num. bigint type

## Demenstion table: using python script :"get_iata_code_from_wiki.py" extract from iata code wiki website.

### 'iata_id' is 3-alpha letter , string type.

### 'airport' is airport name, string type.

### 'location' is country name or city name, string type.




#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

1. tool sets: pandas ,pyspark,pyspark.sql, bs4, BeautifulSoup,requests, csv,time
2. I think the data should be updated every 6 months, because more people immigration frequently.
3. if the data was increased by 100x. use AWS S3 as stage store. and run the spark in the AWS cloud cluster to accelerate the loading data speed and caculation.
4. if the data populates a dashboard that must be updated on a daily basis by 7am every day, it is better to use airflow to make populates automative.
5. if The database needed to be accessed by 100+ people. it is better to use Cassandra database.